### Pasar de la base de datos a diccionario

In [1]:
#Cargamos el csv con los datos
import pandas as pd
df = pd.read_csv("base.csv", sep = ',',encoding='latin-1')

In [2]:
#Creación de función que genera el diccionario necesario para ingresar al algoritmo
def diccionario_lugares(df,fv):
    """
    Función para generar un diccinario con los lugares que visita una fuerza de venta.
    Args:
        df (dataFrame): DataFrame con la base de datos
        fv (int): Fuerza de venta del que queremos generar el diccionario.
    Returns:
        lugares(diccionario): incluye los puntos que debe visitar con sus coordenas. 
    """
    df2 = df[(df.fza_ventas == fv)]
    df2['lugar'] = range(1, len(df2) + 1)
    df2 = df2.append({'lugar' : 'base' , 'lat_destino' : df2.iloc[0]['lat_origen'],'lon_destino': df2.iloc[0]['lon_origen'] } , ignore_index=True)
    df3 = df2[['lugar','lat_destino','lon_destino']].copy()
    lugares = {a:(b,c)for a, b, c in df3.values}
    return lugares

In [3]:
lugares = diccionario_lugares(df,94459)
#Podemos probar con estas fuerza de venta:
# 94459- 6 lugares, Nuevo león
# 37831- 30 lugares, Jalisco
#  14738- 58 lugares, Ciudad de México

/home/miguelmillan13/Documentos/Programas/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [4]:
lugares

{1: (25.671909600000003, -100.1774657),
 2: (25.664719, -100.1628501),
 3: (25.672287100000002, -100.1651263),
 4: (25.678590399999997, -100.16507790000001),
 5: (25.68714417, -100.1897223),
 6: (25.6778829, -100.1665256),
 'base': (25.6626, -100.204)}

In [5]:
import math
import random

from pycsa import CoupledAnnealer

try:
    xrange
except NameError:
    xrange = range

In [6]:
def distance(a, b):
    """
    Helper function to calculate the distance between two 
    latitude-longitude coordinates.
    """
    R = 6371  # radius of Earth (miles)
    lat1, lon1 = math.radians(a[0]), math.radians(a[1])
    lat2, lon2 = math.radians(b[0]), math.radians(b[1])
    return math.acos(math.sin(lat1) * math.sin(lat2) +
                     math.cos(lat1) * math.cos(lat2) * 
                     math.cos(lon1 - lon2)) * R

# Create the distance matrix between the cities.
distance_matrix = {}
for ka, va in lugares.items():
    distance_matrix[ka] = {}
    for kb, vb in lugares.items():
        if kb == ka:
            distance_matrix[ka][kb] = 0.0
        else:
            distance_matrix[ka][kb] = distance(va, vb)

In [7]:
def probe(positions, tgen):
    """
    Swap two cities in the route.
    
    Note that `tgen` (the generation temperature) is ignored here.
    In general, you can use `tgen` to adjust the variance of
    the probing jumps as the algorithm progress.
    """
    a = random.randint(0, len(positions) - 1)
    b = random.randint(0, len(positions) - 1)
    positions[a], positions[b] = positions[b], positions[a]
    return positions


def target(positions):
    """
    Calculates the length of the route.
    """
    e = 0
    for i in xrange(len(positions)):
        e += distance_matrix[positions[i-1]][positions[i]]
    return e


In [8]:
n_annealers = 10  # the number of coupled annealers

init_state = list(lugares.keys())
random.shuffle(init_state)

# Initialize the CSA process.
annealer = CoupledAnnealer(
    target, 
    probe, 
    initial_state=[init_state] * n_annealers,
    steps=200,  # You probably want to set this a lot higher, like 10,000
    processes=1,   # Only use more than 1 process if the target function is costly to compute
    n_annealers=n_annealers,
    tacc_initial=1000.0,
    verbose=1)

# Beging the annealing.
annealer.anneal()

Step      5, Energy 11.9253, Elapsed time 0.00 secs
Updated acceptance temp 950.000000
Updated generation temp 0.010000

Step     10, Energy 12.3654, Elapsed time 0.00 secs
Updated acceptance temp 902.500000
Updated generation temp 0.010000

Step     15, Energy 12.3654, Elapsed time 0.00 secs
Updated acceptance temp 857.375000
Updated generation temp 0.010000

Step     20, Energy 12.3654, Elapsed time 0.00 secs
Updated acceptance temp 814.506250
Updated generation temp 0.010000

Step     25, Energy 12.3654, Elapsed time 0.00 secs
Updated acceptance temp 773.780937
Updated generation temp 0.010000

Step     30, Energy 12.6251, Elapsed time 0.00 secs
Updated acceptance temp 735.091891
Updated generation temp 0.009999

Step     35, Energy 12.3654, Elapsed time 0.00 secs
Updated acceptance temp 698.337296
Updated generation temp 0.009999

Step     40, Energy 12.3654, Elapsed time 0.01 secs
Updated acceptance temp 663.420431
Updated generation temp 0.009999

Step     45, Energy 12.3654, Ela

In [9]:
# Get the best result from all `n_annealers`.
energy, state = annealer.get_best()

# Slide the list of cities until NYC is first.
while state[0] != 'base':
    state = state[1:] + state[:1]

print()
print("%i km route:" % energy)
for city in state:
    print("\t", city)


12 km route:
	 base
	 5
	 3
	 6
	 4
	 1
	 2
